In [2]:
import requests
import base64

CLIENT_ID = 'ce9d2a5314e043bfb80e205a58546f34'
CLIENT_SECRET = '611fa94102ba41a49db5b0e1c5aceb5a' 

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64= base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {'Authorization': f'Basic {client_credentials_base64.decode()}'}
data = {'grant_type': 'client_credentials'}

response = requests.post(token_url, data = data, headers = headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print('Access token obtained successfully')
else:
    print('Error obtaining access token')
    exit()

Access token obtained successfully


In [7]:
pip install spotipy


   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   --

In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)
    
    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields ='items(track(id, name, artists, album(id, name)))')
    
    # Extract relevant information and store in a list of dictionaries
    music_data = [] 
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_id = track['id']
        track_name = track['name']
        artists = ','.join([artist['name'] for artist in track['artists']])
        album_id = track['album']['id']
        album_name = track['album']['name']
        
        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None
    
        #we want the release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None
        # we want the popularity of the track 
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity =  track_info['popularity'] if track_info else None
        except:
            popularity = None
            
        #Add additional track information to the track data
            track_data = {
            'Track ID': track_id,
            'Track Name': track_name,
            'Artists': artists,
            'Album ID': album_id,
            'Album Name': album_name,
            'Release Date': release_date,
            'Popularity': popularity,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
        }

        music_data.append(track_data)
    
    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [4]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                  Track ID                             Track Name  \
0   6AI3ezQ4o3HUoP6Dhudph3                            Not Like Us   
1   3WOhcATHxK2SLNeP5W3v1v          Guess featuring Billie Eilish   
2   0OA00aPt3BV10qeMIs3meW                              Big Dawgs   
3   331l3xABO0HMr1Kkyh2LZq                     I Don't Wanna Wait   
4   7221xIgOnuakPdLqT0F3nP  I Had Some Help (Feat. Morgan Wallen)   
..                     ...                                    ...   
95  6UN73IYd0hZxLi8wFPMQij                                Players   
96  5ZJGv7aGdIr9IGpxzSG18T         Everybody (feat. Lil Uzi Vert)   
97  2UerDbZznc07mmGMD3WfHH                                 it boy   
98  7sliFe6W30tPBPh6dvZsIH                             Area Codes   
99  7mobUfp1aL8A6CdugCMWft                                   BOTH   

                      Artists                Album ID  \
0              Kendrick Lamar  5JjnoGJyOxfSZUZtk2rRwZ   
1    Charli xcx,Billie Eilish  3ThlxfLSy4bfKzxWqmC7VN   


In [5]:
print(music_df.isnull().sum())

Track ID            0
Track Name          0
Artists             0
Album ID            0
Album Name          0
Release Date        0
Popularity          0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

data = music_df

In [8]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness',
                         'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [9]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date
    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)  # Adding 1 to avoid division by zero
    
    return weight

In [10]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations = 5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' is not in dataset. Please check for valid song name")
        return
    
    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]
    
    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)
    
    # Get the indices of the most similar songs
    similarity_scores_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]
    
    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similarity_scores_indices][['Track Name', 'Artists', 'Album Name', 
                                                                             'Release Date', 'Popularity']]
    return content_based_recommendations
    

In [11]:
input_song_name = 'Not Like Us'
recommendations = content_based_recommendations(input_song_name, num_recommendations = 5)
print(f"content based recommended these songs for'{input_song_name}':")
print(recommendations)

content based recommended these songs for'Not Like Us':
            Track Name         Artists  \
26         Lovin On Me     Jack Harlow   
49                Fuel      Eminem,JID   
29            euphoria  Kendrick Lamar   
48        Belly Dancer    Imanbek,BYOR   
69  Sunshine (My Girl)            Wuki   

                                 Album Name Release Date  Popularity  
26                              Lovin On Me   2023-11-10          82  
49  The Death of Slim Shady (Coup De Grâce)   2024-07-12          77  
29                                 euphoria   2024-04-30          81  
48                             Belly Dancer   2022-02-18          77  
69                       Sunshine (My Girl)   2023-09-01          74  


In [12]:
# a function to get hybrid recommendations based on music features
def hybrid_recommendations(input_song_name, num_recommendations = 5, alpha = 0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' is not in the dataset. Please check for valid song name")
        return

    #calling the content based recommendation
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)
    
    #Defining the popularity score
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]
    
    #Calculating the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])
    
    #Creating a new entry dataframe
    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
        })
    
    #Concatenating the content based recommendation and the new entry dataframe
    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index= True)
    
    #Sorting the values in descending order by the popularity
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)
    
    #Eliminating the input song name from the recommendations list
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]
    
    return hybrid_recommendations 



In [13]:
input_song_name = 'Not Like Us'
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended these songs for'{input_song_name}':")
print(recommendations)

Hybrid recommended these songs for'Not Like Us':
           Track Name         Artists  \
0         Lovin On Me     Jack Harlow   
2            euphoria  Kendrick Lamar   
1                Fuel      Eminem,JID   
3        Belly Dancer    Imanbek,BYOR   
4  Sunshine (My Girl)            Wuki   

                                Album Name Release Date  Popularity  
0                              Lovin On Me   2023-11-10        82.0  
2                                 euphoria   2024-04-30        81.0  
1  The Death of Slim Shady (Coup De Grâce)   2024-07-12        77.0  
3                             Belly Dancer   2022-02-18        77.0  
4                       Sunshine (My Girl)   2023-09-01        74.0  
